*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# The Microsoft Toolkit of Multi-Task Deep Neural Networks for Natural Language Understanding


## Summary

MT-DNN is an open-source natural language understanding (NLU) toolkit that makes it easy for researchers and developers to train customized deep learning models. Built upon PyTorch and Transformers, MT-DNN is designed to facilitate rapid
customization for a broad spectrum of NLU tasks, using a variety of objectives (classification, regression, structured prediction) and text encoders (e.g., RNNs, BERT, RoBERTa, UniLM). A unique feature of MT-DNN is its built-in support for robust and transferable learning using the adversarial multi-task learning paradigm. To enable efficient production deployment, MT-DNN supports multitask knowledge distillation, which can substantially compress a deep neural model without significant performance drop. We demonstrate the effectiveness of MT-DNN on a wide range of NLU applications across general and biomedical domains. The pip installable package and pretrained models will be publicly available at https://github.com/microsoft/mt-dnn.

### Design

MT-DNN is designed for modularity, flexibility, and ease of use. These modules are built upon PyTorch (Paszke et al., 2019) and Transformers (Wolf
et al., 2019), allowing the use of the SOTA pretrained models, e.g., BERT (Devlin et al., 2019), RoBERTa (Liu et al., 2019c) and UniLM (Dong
et al., 2019). The unique attribute of this package is a flexible interface for adversarial multi-task fine-tuning and knowledge distillation, so that researchers and developers can build large SOTA NLU models and then compress them to small ones
for online deployment.The overall workflow and system architecture are shown in figures 1 and 3 respectively.


![Workflow Design](https://nlpbp.blob.core.windows.net/images/mt-dnn2.JPG)

The above figure shows workflow of MT-DNN: train a neural language model on a large amount of unlabeled raw text
to obtain general contextual representations; then finetune the learned contextual representation on downstream tasks, e.g. GLUE (Wang et al., 2018); lastly, distill this large model to a lighter one for online deployment. In the later two phrases, we can leverage powerful multi-task learning and adversarial training to further improve performance.

## Architecture

![overall_arch](https://nlpbp.blob.core.windows.net/images/mt-dnn.png)
The figure above shows the overall system architecture. The lower layers are shared across all tasks while the top layers are taskspecific. The input X (either a sentence or a set of sentences) is first represented as a sequence of embedding
vectors, one for each word, in l1. Then the encoder, e.g a Transformer or recurrent neural network (LSTM) model,
captures the contextual information for each word and generates the shared contextual embedding vectors in l2.
Finally, for each task, additional task-specific layers generate task-specific representations, followed by operations
necessary for classification, similarity scoring, or relevance ranking. In case of adversarial training, we perturb
embeddings from the lexicon encoder and then add an extra loss term during the training. Note that for the
inference phrase, it does not require perturbations.

## Introduction
In this notebook, we fine-tune and evaluate MT-DNN models on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.  

### Running Time

This is a __computationally intensive__ notebook that runs on the entire MNLI dataset for match and mismatched datasets for training, development and test.  

The table below provides some reference running time on a GPU machine.  

|Dataset|MULTI_GPU_ON|Machine Configurations|Running time|
|:------|:---------|:----------------------|:------------|
|MultiNLI|True|4 NVIDIA Tesla K80 GPUs, 24GB GPU memory| ~ 20 hours |

If you run into `CUDA out-of-memory error` or the jupyter kernel dies constantly, try reducing the `BATCH_SIZE` and `MAX_SEQ_LEN` in `MTDNNConfig`, but note that model performance may be compromised.


## PyTorch Setup 

In [1]:
# uninstall the default 1.4.0 and run the following command (due to the config of our dev box)
# !pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html


### Text Classification of MultiNLI Sentences using MT-DNN

This notebook utilizes the pip installable package that implements the Multi-Task Deep Neural Network Toolkit (MTDNN) for Natural Language Understanding. It's recommended to run this notebook on GPU machines as it's very computationally intensive.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import shutil
import sys
from datetime import datetime

import pandas as pd
import torch
import tensorflow.io.gfile as gfile

from mtdnn.common.types import EncoderModelType
from mtdnn.configuration_mtdnn import MTDNNConfig
from mtdnn.data_builder_mtdnn import MTDNNDataBuilder
from mtdnn.modeling_mtdnn import MTDNNModel
from mtdnn.process_mtdnn import MTDNNDataProcess
from mtdnn.tasks.config import MTDNNTaskDefs
from mtdnn.tokenizer_mtdnn import MTDNNTokenizer

## Define Configuration, Tasks and Model Objects

In [3]:
DEBUG = False

In [4]:
# Define Configuration, Tasks and Model Objects
ROOT_DIR = 'gs://cs330'
MODEL_ID = datetime.now().strftime('%m%d%H%M')
OUTPUT_DIR = os.path.join(ROOT_DIR, 'checkpoint', MODEL_ID)

DATA_DIR = "../../../glue_data/" if not DEBUG else "../../../debug_data/"
TASK_DATA_DIRS = {
    'qqp': os.path.join(DATA_DIR, "QQP"),
    'mnli': os.path.join(DATA_DIR, "MNLI"),
    'sst': os.path.join(DATA_DIR, "SST-2"),
    'mrpc': os.path.join(DATA_DIR, "MRPC")
    }

In [5]:
# Training parameters
BATCH_SIZE = 16
MULTI_GPU_ON = False
MAX_SEQ_LEN = 128
NUM_EPOCHS = 5

In [6]:
TASK_DATA_DIRS

{'qqp': '../../../glue_data/QQP',
 'mnli': '../../../glue_data/MNLI',
 'sst': '../../../glue_data/SST-2',
 'mrpc': '../../../glue_data/MRPC'}

Exploring the location for our data to be downloaded, model to be checkpointed and logs to be dumped

In [7]:
print(OUTPUT_DIR)

gs://cs330/checkpoint/11031951


### Define a Configuration Object 

Create a model configuration object, `MTDNNConfig`, with the necessary parameters to initialize the MT-DNN model. Initialization without any parameters will default to a similar configuration that initializes a BERT model. 

In [8]:
config = MTDNNConfig(batch_size=BATCH_SIZE, 
                     max_seq_len=MAX_SEQ_LEN, 
                     multi_gpu_on=MULTI_GPU_ON)


### Create Task Definition Object  

Define the task parameters to train for and initialize an `MTDNNTaskDefs` object. Create a task parameter dictionary. Definition can be a single or multiple tasks to train.  `MTDNNTaskDefs` can take a python dict, yaml or json file with task(s) defintion.

The data source directory is the path of data downloaded and extracted above using `download_tsv_files_and_extract` which is the `MNLI` dir under the `DATA_DIR` temporary directory.    

The data source has options that are set to drive each task pre-processing; `data_process_opts`


In [9]:
default_data_process_opts = {"header": True, "is_train": True, "multi_snli": False,}
default_split_names = ["train", "dev", "test"]
tasks_params = {
    "mnli": {
        "data_format": "PremiseAndOneHypothesis",
        "encoder_type": "BERT",
        "dropout_p": 0.3,
        "enable_san": True,
        "labels": ["contradiction", "neutral", "entailment"],
        "metric_meta": ["ACC"],
        "loss": "CeCriterion",
        "kd_loss": "MseCriterion",
        "n_class": 3,
        "split_names": [
            "train",
            "dev_matched",
            "dev_mismatched",
            "test_matched",
            "test_mismatched",
        ],
        "data_source_dir": TASK_DATA_DIRS['mnli'],
        "data_process_opts": default_data_process_opts,
        "task_type": "Classification",
    },
    "mrpc": {
                "task_name": "mrpc",
                "data_format": "PremiseAndOneHypothesis",
                "encoder_type": "BERT",
                "enable_san": True,
                "metric_meta": ["ACC", "F1"],
                "loss": "CeCriterion",
                "kd_loss": "MseCriterion",
                "n_class": 2,
                "split_names": default_split_names,
                "data_source_dir": TASK_DATA_DIRS['mrpc'],
                "data_process_opts": default_data_process_opts,
                "task_type": "Classification",
            },
    "sst": {
                "task_name": "sst",
                "data_format": "PremiseOnly",
                "encoder_type": "BERT",
                "enable_san": False,
                "metric_meta": ["ACC"],
                "loss": "CeCriterion",
                "kd_loss": "MseCriterion",
                "n_class": 2,
                "split_names": default_split_names,
                "data_source_dir": TASK_DATA_DIRS['sst'],
                "data_process_opts": default_data_process_opts,
                "task_type": "Classification",
            },
    "qqp": {
        "task_name": "qqp",
        "data_format": "PremiseAndOneHypothesis",
        "encoder_type": "BERT",
        "enable_san": True,
        "metric_meta": ["ACC", "F1"],
        "loss": "CeCriterion",
        "kd_loss": "MseCriterion",
        "n_class": 2,
        "split_names": default_split_names,
        "data_source_dir": TASK_DATA_DIRS['qqp'],
        "data_process_opts": default_data_process_opts,
        "task_type": "Classification",
    }
}

# Define the tasks
task_defs = MTDNNTaskDefs(tasks_params)

11/03/2020 07:51:29 - mtdnn.tasks.config - INFO - Mapping Task attributes
11/03/2020 07:51:29 - mtdnn.tasks.config - INFO - Mapping Task attributes
11/03/2020 07:51:29 - mtdnn.tasks.config - INFO - Mapping Task attributes
11/03/2020 07:51:29 - mtdnn.tasks.config - INFO - Mapping Task attributes
11/03/2020 07:51:29 - mtdnn.tasks.config - INFO - Configured task definitions - ['mnli', 'mrpc', 'sst', 'qqp']



### Create the MTDNN Data Tokenizer Object  

Create a data tokenizing object, `MTDNNTokenizer`. Based on the model initial checkpoint, it wraps around the model's Huggingface transformers library to encode the data to MT-DNN format. This becomes the input to the data building stage.  


In [10]:
tokenizer = MTDNNTokenizer(do_lower_case=True)

#### Testing out the Tokenizer encode function on a sample text
`tokenizer.encode("What NLP toolkit do you recommend", "MT-DNN is a fantastic toolkit")`

In [11]:
# single sentence
tokenizer.encode("What NLP toolkit do you recommend")

([101, 2054, 17953, 2361, 6994, 23615, 2079, 2017, 16755, 102, 102],
 None,
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [12]:
# sentence pair
print(tokenizer.encode("What NLP toolkit do you recommend", "MT-DNN is a fantastic toolkit"))

([101, 2054, 17953, 2361, 6994, 23615, 2079, 2017, 16755, 102, 11047, 1011, 1040, 10695, 2003, 1037, 10392, 6994, 23615, 102], None, [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])


## Data Preprocessing

### Create the Data Builder Object  

Create a data preprocessing object, `MTDNNDataBuilder`. This class is responsible for converting the data into the MT-DNN format depending on the task.  
 

Define a data builder that handles the creating of each task's vectorized data utilizing the model tokenizer. This will build out the vectorized data needed for creating the training, test and development PyTorch dataloaders

In [13]:
task_defs.data_paths_map

{'mnli': {'data_paths': ['../../../glue_data/MNLI/train.tsv',
   '../../../glue_data/MNLI/dev_matched.tsv',
   '../../../glue_data/MNLI/dev_mismatched.tsv',
   '../../../glue_data/MNLI/test_matched.tsv',
   '../../../glue_data/MNLI/test_mismatched.tsv'],
  'data_opts': {'header': True, 'is_train': True, 'multi_snli': False}},
 'mrpc': {'data_paths': ['../../../glue_data/MRPC/train.tsv',
   '../../../glue_data/MRPC/dev.tsv',
   '../../../glue_data/MRPC/test.tsv'],
  'data_opts': {'header': True, 'is_train': True, 'multi_snli': False}},
 'sst': {'data_paths': ['../../../glue_data/SST-2/train.tsv',
   '../../../glue_data/SST-2/dev.tsv',
   '../../../glue_data/SST-2/test.tsv'],
  'data_opts': {'header': True, 'is_train': True, 'multi_snli': False}},
 'qqp': {'data_paths': ['../../../glue_data/QQP/train.tsv',
   '../../../glue_data/QQP/dev.tsv',
   '../../../glue_data/QQP/test.tsv'],
  'data_opts': {'header': True, 'is_train': True, 'multi_snli': False}}}

In [14]:
## Load and build data
data_builder = MTDNNDataBuilder(
    tokenizer=tokenizer,
    task_defs=task_defs,
    data_dir='.', #DATA_SOURCE_DIR,
    canonical_data_suffix="canonical_data",
    dump_rows=True,
)

## Build data to MTDNN Format
## Iterable of each specific task and processed data
vectorized_data = data_builder.vectorize()

11/03/2020 07:51:40 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 40000 samples for mnli at ./canonical_data/mnli_train.tsv
11/03/2020 07:51:40 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 5000 samples for mnli at ./canonical_data/mnli_dev_matched.tsv
11/03/2020 07:51:40 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 5000 samples for mnli at ./canonical_data/mnli_dev_mismatched.tsv
11/03/2020 07:51:40 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9796 samples for mnli at ./canonical_data/mnli_test_matched.tsv
11/03/2020 07:51:41 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 9847 samples for mnli at ./canonical_data/mnli_test_mismatched.tsv
11/03/2020 07:51:41 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 3668 samples for mrpc at ./canonical_data/mrpc_train.tsv
11/03/2020 07:51:41 - mtdnn.data_builder_mtdnn - INFO - Sucessfully loaded and built 408 samples for mrpc at ./cano

Building Data For Premise and One Hypothesis: 40000it [00:30, 1315.17it/s]

11/03/2020 07:52:13 - mtdnn.data_builder_mtdnn - INFO - Saving data to ./canonical_data/bert_base_uncased/mnli_train.json



Saving Data For PremiseAndOneHypothesis: 100%|██████████| 40000/40000 [00:01<00:00, 29401.70it/s]

mnli_dev_matched
11/03/2020 07:52:15 - mtdnn.data_builder_mtdnn - INFO - Building Data For 'MNLI DEV MATCHED' Task


...


Saving Data For PremiseAndOneHypothesis: 100%|██████████| 10000/10000 [00:00<00:00, 49334.07it/s]

qqp_test
11/03/2020 07:53:18 - mtdnn.data_builder_mtdnn - INFO - Building Data For 'QQP TEST' Task



Building Data For Premise and One Hypothesis: 390965it [04:40, 1392.72it/s]

11/03/2020 07:58:00 - mtdnn.data_builder_mtdnn - INFO - Saving data to ./canonical_data/bert_base_uncased/qqp_test.json



Saving Data For PremiseAndOneHypothesis: 100%|██████████| 390965/390965 [00:09<00:00, 39116.24it/s]


### Create the Data Processing Object  

Create a data preprocessing object, `MTDNNDataProcess`. This creates the training, test and development PyTorch dataloaders needed for training and testing. We also need to retrieve the necessary training options required to initialize the model correctly, for all tasks.  

Define a data process that handles creating the training, test and development PyTorch dataloaders

In [15]:
# Make the Data Preprocess step and update the config with training data updates
data_processor = MTDNNDataProcess(
    config=config, task_defs=task_defs, vectorized_data=vectorized_data
)

11/03/2020 07:58:22 - mtdnn.process_mtdnn - INFO - Starting to process the training data sets
11/03/2020 07:58:22 - mtdnn.process_mtdnn - INFO - Loading mnli_train as task 0
11/03/2020 07:58:22 - mtdnn.dataset_mtdnn - INFO - Loaded 39908 samples out of 40000
11/03/2020 07:58:22 - mtdnn.process_mtdnn - INFO - Loading mrpc_train as task 1
11/03/2020 07:58:22 - mtdnn.dataset_mtdnn - INFO - Loaded 3668 samples out of 3668
11/03/2020 07:58:22 - mtdnn.process_mtdnn - INFO - Loading sst_train as task 2
11/03/2020 07:58:22 - mtdnn.dataset_mtdnn - INFO - Loaded 8000 samples out of 8000
11/03/2020 07:58:22 - mtdnn.process_mtdnn - INFO - Loading qqp_train as task 3
11/03/2020 07:58:22 - mtdnn.dataset_mtdnn - INFO - Loaded 39986 samples out of 40000
11/03/2020 07:58:22 - mtdnn.process_mtdnn - INFO - Starting to process the testing data sets
11/03/2020 07:58:22 - mtdnn.process_mtdnn - INFO - Loading mnli_dev_matched as task 0
11/03/2020 07:58:22 - mtdnn.dataset_mtdnn - INFO - Loaded 5000 samples ou

### Instantiate the MTDNN Model

Now we can go ahead and create an `MTDNNModel` model

In [16]:
model = MTDNNModel(
    config,
    task_defs,
    data_processor=data_processor,
    pretrained_model_name="bert-base-uncased",
    output_dir=OUTPUT_DIR,
)

wandb: Currently logged in as: feifang24 (use `wandb login --relogin` to force relogin)


idx: 0, number of task labels: 3
idx: 1, number of task labels: 2
idx: 2, number of task labels: 2
idx: 3, number of task labels: 2


In [17]:
for idx, dataset in enumerate(model.test_datasets_list):
    print(idx, dataset)

0 mnli_matched
1 mnli_mismatched
2 mrpc
3 sst
4 qqp


In [18]:
for idx, task in enumerate(model.tasks):
    print(idx, task)

0 mnli
1 mrpc
2 sst
3 qqp


## Model Finetuning, Prediction and Evaluation

### Fit and finetune model on five epochs and predict using the training and test  

At this point the MT-DNN model allows us to fit to the model and create predictions. The fit takes an optional `epochs` parameter that overwrites the epochs set in the `MTDNNConfig` object. 

In [19]:
model.fit(epochs=NUM_EPOCHS)

11/03/2020 07:58:40 - mtdnn.modeling_mtdnn - INFO - Total number of params: 109489161
11/03/2020 07:58:40 - mtdnn.modeling_mtdnn - INFO - At epoch 1
11/03/2020 07:58:40 - mtdnn.modeling_mtdnn - INFO - Amount of data to go over: 5725
11/03/2020 07:58:41 - mtdnn.modeling_mtdnn - INFO - Task - [ 3] Updates - [     1] Training Loss - [0.70013] Time Remaining - [0:36:40]


/home/feifang/mtl-epistemic-uncertainty/microsoft-mt-dnn/mtdnn/common/bert_optim.py:144: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)


11/03/2020 08:00:55 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [   500] Training Loss - [0.89386] Time Remaining - [0:23:30]
11/03/2020 08:03:19 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [  1000] Training Loss - [0.84969] Time Remaining - [0:21:57]
11/03/2020 08:05:39 - mtdnn.modeling_mtdnn - INFO - Task - [ 3] Updates - [  1500] Training Loss - [0.81297] Time Remaining - [0:19:38]
11/03/2020 08:07:56 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [  2000] Training Loss - [0.77324] Time Remaining - [0:17:14]
11/03/2020 08:10:18 - mtdnn.modeling_mtdnn - INFO - Task - [ 3] Updates - [  2500] Training Loss - [0.73841] Time Remaining - [0:14:59]
11/03/2020 08:12:39 - mtdnn.modeling_mtdnn - INFO - Task - [ 0] Updates - [  3000] Training Loss - [0.71030] Time Remaining - [0:12:41]
11/03/2020 08:14:55 - mtdnn.modeling_mtdnn - INFO - Task - [ 3] Updates - [  3500] Training Loss - [0.68737] Time Remaining - [0:10:19]
11/03/2020 08:17:13 - mtdnn.modeling_mtdnn - INF

11/03/2020 08:56:08 - mtdnn.modeling_mtdnn - INFO - predicting 200
11/03/2020 08:56:12 - mtdnn.modeling_mtdnn - INFO - predicting 300
11/03/2020 08:56:16 - mtdnn.modeling_mtdnn - INFO - predicting 400
11/03/2020 08:56:19 - mtdnn.modeling_mtdnn - INFO - predicting 500
11/03/2020 08:56:23 - mtdnn.modeling_mtdnn - INFO - predicting 600
11/03/2020 08:56:24 - mtdnn.modeling_mtdnn - INFO - Task mnli_mismatched -- Dev loss: 0.603
11/03/2020 08:56:24 - mtdnn.modeling_mtdnn - INFO - Task mnli_mismatched -- Dev ACC: 77.840
11/03/2020 08:56:24 - mtdnn.modeling_mtdnn - INFO - Evaluating on dev ds 2: MRPC
11/03/2020 08:56:24 - mtdnn.modeling_mtdnn - INFO - predicting 0
11/03/2020 08:56:27 - mtdnn.modeling_mtdnn - INFO - Task mrpc -- Dev loss: 0.466
11/03/2020 08:56:27 - mtdnn.modeling_mtdnn - INFO - Task mrpc -- Dev ACC: 76.961
11/03/2020 08:56:27 - mtdnn.modeling_mtdnn - INFO - Task mrpc -- Dev F1: 83.392
11/03/2020 08:56:27 - mtdnn.modeling_mtdnn - INFO - Evaluating on dev ds 3: SST
11/03/2020 08

11/03/2020 09:24:44 - mtdnn.modeling_mtdnn - INFO - Task qqp -- Dev F1: 81.509
11/03/2020 09:24:44 - mtdnn.modeling_mtdnn - INFO - Dev loss: 0.530181315788358
11/03/2020 09:24:44 - mtdnn.modeling_mtdnn - INFO - Saving mt-dnn model to gs://cs330/checkpoint/11031951/model_3.pt
11/03/2020 09:25:03 - mtdnn.modeling_mtdnn - INFO - model saved to gs://cs330/checkpoint/11031951/model_3.pt
11/03/2020 09:25:03 - mtdnn.modeling_mtdnn - INFO - At epoch 4
11/03/2020 09:25:03 - mtdnn.modeling_mtdnn - INFO - Amount of data to go over: 5725
11/03/2020 09:26:32 - mtdnn.modeling_mtdnn - INFO - Task - [ 2] Updates - [ 17500] Training Loss - [0.43598] Time Remaining - [0:24:40]
11/03/2020 09:28:46 - mtdnn.modeling_mtdnn - INFO - Task - [ 3] Updates - [ 18000] Training Loss - [0.43111] Time Remaining - [0:22:09]
11/03/2020 09:31:02 - mtdnn.modeling_mtdnn - INFO - Task - [ 3] Updates - [ 18500] Training Loss - [0.42675] Time Remaining - [0:19:55]
11/03/2020 09:33:17 - mtdnn.modeling_mtdnn - INFO - Task - [

11/03/2020 10:18:36 - mtdnn.modeling_mtdnn - INFO - predicting 300
11/03/2020 10:18:40 - mtdnn.modeling_mtdnn - INFO - predicting 400
11/03/2020 10:18:44 - mtdnn.modeling_mtdnn - INFO - predicting 500
11/03/2020 10:18:47 - mtdnn.modeling_mtdnn - INFO - predicting 600
11/03/2020 10:18:49 - mtdnn.modeling_mtdnn - INFO - Task mnli_matched -- Dev loss: 0.782
11/03/2020 10:18:49 - mtdnn.modeling_mtdnn - INFO - Task mnli_matched -- Dev ACC: 78.700
11/03/2020 10:18:49 - mtdnn.modeling_mtdnn - INFO - Evaluating on dev ds 1: MNLI_MISMATCHED
11/03/2020 10:18:49 - mtdnn.modeling_mtdnn - INFO - predicting 0
11/03/2020 10:18:52 - mtdnn.modeling_mtdnn - INFO - predicting 100
11/03/2020 10:18:56 - mtdnn.modeling_mtdnn - INFO - predicting 200
11/03/2020 10:19:00 - mtdnn.modeling_mtdnn - INFO - predicting 300
11/03/2020 10:19:04 - mtdnn.modeling_mtdnn - INFO - predicting 400
11/03/2020 10:19:07 - mtdnn.modeling_mtdnn - INFO - predicting 500
11/03/2020 10:19:11 - mtdnn.modeling_mtdnn - INFO - predicting

### Evaluation and Prediction
Perform inference using the last/best checkpointed model. The epochs are 1-indexed.

With 5 epochs, the *last* model would be `model_5.pt`.

Identify *best* checkpoint from wandb logs.

In [20]:
BEST_CHECKPOINT_EPOCH = 3 # set to None if want to default to last

In [21]:
EVAL_CHECKPOINT = NUM_EPOCHS if BEST_CHECKPOINT_EPOCH is None else BEST_CHECKPOINT_EPOCH

In [23]:
# can replace OUTPUT_DIR with arbitrary model_dir as long as it's the same type of network
model.predict(trained_model_chckpt=f"{OUTPUT_DIR}/model_{EVAL_CHECKPOINT}.pt")

11/03/2020 10:26:27 - mtdnn.modeling_mtdnn - INFO - Running predictions using: gs://cs330/checkpoint/11031951/model_3.pt. This may take 3 minutes.
11/03/2020 10:29:35 - mtdnn.modeling_mtdnn - INFO - Checkpoint loaded.
11/03/2020 10:29:35 - mtdnn.modeling_mtdnn - INFO - predicting 0
11/03/2020 10:29:39 - mtdnn.modeling_mtdnn - INFO - predicting 100
11/03/2020 10:29:43 - mtdnn.modeling_mtdnn - INFO - predicting 200
11/03/2020 10:29:46 - mtdnn.modeling_mtdnn - INFO - predicting 300
11/03/2020 10:29:50 - mtdnn.modeling_mtdnn - INFO - predicting 400
11/03/2020 10:29:54 - mtdnn.modeling_mtdnn - INFO - predicting 500
11/03/2020 10:29:57 - mtdnn.modeling_mtdnn - INFO - predicting 600
11/03/2020 10:30:01 - mtdnn.modeling_mtdnn - INFO - predicting 700
11/03/2020 10:30:05 - mtdnn.modeling_mtdnn - INFO - predicting 800
11/03/2020 10:30:09 - mtdnn.modeling_mtdnn - INFO - predicting 900
11/03/2020 10:30:13 - mtdnn.modeling_mtdnn - INFO - predicting 1000
11/03/2020 10:30:17 - mtdnn.modeling_mtdnn - I

11/03/2020 10:35:28 - mtdnn.modeling_mtdnn - INFO - predicting 8100
11/03/2020 10:35:31 - mtdnn.modeling_mtdnn - INFO - predicting 8200
11/03/2020 10:35:34 - mtdnn.modeling_mtdnn - INFO - predicting 8300
11/03/2020 10:35:37 - mtdnn.modeling_mtdnn - INFO - predicting 8400
11/03/2020 10:35:40 - mtdnn.modeling_mtdnn - INFO - predicting 8500
11/03/2020 10:35:43 - mtdnn.modeling_mtdnn - INFO - predicting 8600
11/03/2020 10:35:46 - mtdnn.modeling_mtdnn - INFO - predicting 8700
11/03/2020 10:35:49 - mtdnn.modeling_mtdnn - INFO - predicting 8800
11/03/2020 10:35:51 - mtdnn.modeling_mtdnn - INFO - predicting 8900
11/03/2020 10:35:54 - mtdnn.modeling_mtdnn - INFO - predicting 9000
11/03/2020 10:35:57 - mtdnn.modeling_mtdnn - INFO - predicting 9100
11/03/2020 10:36:00 - mtdnn.modeling_mtdnn - INFO - predicting 9200
11/03/2020 10:36:03 - mtdnn.modeling_mtdnn - INFO - predicting 9300
11/03/2020 10:36:06 - mtdnn.modeling_mtdnn - INFO - predicting 9400
11/03/2020 10:36:08 - mtdnn.modeling_mtdnn - INF

11/03/2020 10:41:20 - mtdnn.modeling_mtdnn - INFO - predicting 20100
11/03/2020 10:41:23 - mtdnn.modeling_mtdnn - INFO - predicting 20200
11/03/2020 10:41:26 - mtdnn.modeling_mtdnn - INFO - predicting 20300
11/03/2020 10:41:29 - mtdnn.modeling_mtdnn - INFO - predicting 20400
11/03/2020 10:41:32 - mtdnn.modeling_mtdnn - INFO - predicting 20500
11/03/2020 10:41:36 - mtdnn.modeling_mtdnn - INFO - predicting 20600
11/03/2020 10:41:39 - mtdnn.modeling_mtdnn - INFO - predicting 20700
11/03/2020 10:41:41 - mtdnn.modeling_mtdnn - INFO - predicting 20800
11/03/2020 10:41:44 - mtdnn.modeling_mtdnn - INFO - predicting 20900
11/03/2020 10:41:47 - mtdnn.modeling_mtdnn - INFO - predicting 21000
11/03/2020 10:41:50 - mtdnn.modeling_mtdnn - INFO - predicting 21100
11/03/2020 10:41:53 - mtdnn.modeling_mtdnn - INFO - predicting 21200
11/03/2020 10:41:56 - mtdnn.modeling_mtdnn - INFO - predicting 21300
11/03/2020 10:41:59 - mtdnn.modeling_mtdnn - INFO - predicting 21400
11/03/2020 10:42:02 - mtdnn.modeli

11/03/2020 10:47:16 - mtdnn.modeling_mtdnn - INFO - predicting 32000
11/03/2020 10:47:19 - mtdnn.modeling_mtdnn - INFO - predicting 32100
11/03/2020 10:47:22 - mtdnn.modeling_mtdnn - INFO - predicting 32200
11/03/2020 10:47:25 - mtdnn.modeling_mtdnn - INFO - predicting 32300
11/03/2020 10:47:28 - mtdnn.modeling_mtdnn - INFO - predicting 32400
11/03/2020 10:47:31 - mtdnn.modeling_mtdnn - INFO - predicting 32500
11/03/2020 10:47:34 - mtdnn.modeling_mtdnn - INFO - predicting 32600
11/03/2020 10:47:37 - mtdnn.modeling_mtdnn - INFO - predicting 32700
11/03/2020 10:47:40 - mtdnn.modeling_mtdnn - INFO - predicting 32800
11/03/2020 10:47:43 - mtdnn.modeling_mtdnn - INFO - predicting 32900
11/03/2020 10:47:46 - mtdnn.modeling_mtdnn - INFO - predicting 33000
11/03/2020 10:47:49 - mtdnn.modeling_mtdnn - INFO - predicting 33100
11/03/2020 10:47:52 - mtdnn.modeling_mtdnn - INFO - predicting 33200
11/03/2020 10:47:55 - mtdnn.modeling_mtdnn - INFO - predicting 33300
11/03/2020 10:47:58 - mtdnn.modeli

11/03/2020 10:53:00 - mtdnn.modeling_mtdnn - INFO - predicting 43900
11/03/2020 10:53:03 - mtdnn.modeling_mtdnn - INFO - predicting 44000
11/03/2020 10:53:05 - mtdnn.modeling_mtdnn - INFO - predicting 44100
11/03/2020 10:53:08 - mtdnn.modeling_mtdnn - INFO - predicting 44200
11/03/2020 10:53:11 - mtdnn.modeling_mtdnn - INFO - predicting 44300
11/03/2020 10:53:14 - mtdnn.modeling_mtdnn - INFO - predicting 44400
11/03/2020 10:53:17 - mtdnn.modeling_mtdnn - INFO - predicting 44500
11/03/2020 10:53:20 - mtdnn.modeling_mtdnn - INFO - predicting 44600
11/03/2020 10:53:23 - mtdnn.modeling_mtdnn - INFO - predicting 44700
11/03/2020 10:53:26 - mtdnn.modeling_mtdnn - INFO - predicting 44800
11/03/2020 10:53:29 - mtdnn.modeling_mtdnn - INFO - predicting 44900
11/03/2020 10:53:31 - mtdnn.modeling_mtdnn - INFO - predicting 45000
11/03/2020 10:53:34 - mtdnn.modeling_mtdnn - INFO - predicting 45100
11/03/2020 10:53:37 - mtdnn.modeling_mtdnn - INFO - predicting 45200
11/03/2020 10:53:40 - mtdnn.modeli

### Results

In [31]:
results = {}
dev_result_files = list(filter(lambda x: x.endswith('.json') and 'dev' in x and str(EVAL_CHECKPOINT) in x, gfile.listdir(OUTPUT_DIR))) 

In [33]:
for d in dev_result_files:
    name =  ' '.join(list(map(str.capitalize, d.split('_')))[:3]) 
    file_name = os.path.join(OUTPUT_DIR, d)
    with gfile.GFile(file_name, 'r') as f: 
        res = json.load(f) 
        results.update(
            {name: {
                'ACCURACY': f"{res['metrics']['ACC']:.3f}"
                }
            }) 
df_results = pd.DataFrame(results)   
df_results

,Mnli Matched Dev,Mnli Mismatched Dev,Mrpc Dev Scores,Qqp Dev Scores,Sst Dev Scores
ACCURACY,78.240,78.480,79.902,86.470,90.300
